In [13]:
import pandas as pd

data = pd.read_csv('data/train.csv', index_col = 0, dtype={'multiple_full_time_jobs': 'boolean', 'combined_multiple_jobs': 'boolean'}).reset_index(drop=True)
strCols = data.select_dtypes(object).columns
for i in strCols:
    data[i] = data[i].str.strip()
#data = data.sample(frac=0.1)
data = data.fillna(True)
data.HIREDT = data.HIREDT.apply(pd.to_datetime, format = "%m/%d/%y")
data['daysexperience'] = (pd.to_datetime('10/7/23', format = "%m/%d/%y") - data.HIREDT ).dt.days
data_num = ['RATE', 'HRSWKD', 'daysexperience', 'multiple_full_time_jobs', 'combined_multiple_jobs']
data_drop = ['AGY', 'HIREDT']#, 'multiple_full_time_jobs', 'combined_multiple_jobs']
data.drop(data_drop, axis=1, inplace=True)

important_features = ['daysexperience', 'RATE', 'RACE', 'SEX', 'MI', 'HRSWKD', 'JC.TITLE']
data_to_onehot = ['NAME', 'MI', 'JOBCLASS', 'JC.TITLE', 'RACE', 'SEX', 'EMPTYPE', 'STATENUM']

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import scipy as sp
#regressor = LinearRegression(n_jobs=-1) np.log10 sp.special.exp10
regressor = RandomForestRegressor(n_estimators=100, verbose=3, random_state=0,n_jobs=-1)
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'infrequent_if_exist', drop='if_binary'), data_to_onehot),
    remainder = 'passthrough',
)
pipe = Pipeline([('transformer', transformer), ('regressor', regressor)])

In [15]:
from sklearn.model_selection import train_test_split
X = data.drop('ANNUAL', axis=1)
y = data.ANNUAL

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   31.8s


building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.4min finished
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2, 3, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished


0.8455870125166478

In [2]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for the Decision Tree Regressor
param_grid = {
    'regressor__max_depth': range(1, 20),
    'regressor__min_samples_split': range(2, 10),
    'regressor__min_samples_leaf': range(1, 10),
}

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the GridSearchCV object
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best R-squared score: {best_score:.2f}")

NameError: name 'pipe' is not defined

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_pred, y_test)

In [ ]:
import pandas as pd

data = pd.read_csv('data/train.csv', index_col = 0, dtype={'multiple_full_time_jobs': 'boolean', 'combined_multiple_jobs': 'boolean'}).reset_index(drop=True)
# Display the first few rows of the dataset
top_rows_str = data.head().to_string()

# Display the summary statistics of the dataset
summary_statistics_str = data.describe().to_string()

# Check the correlation between the features and the target variable
correlations = data.corr()
correlations_str = correlations.to_string()

output_str = f"First few rows of the dataset:\n{top_rows_str}\n\nSummary statistics of the dataset:\n{summary_statistics_str}\n\nCorrelation matrix:\n{correlations_str}"
output_str